In [113]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt
import linecache
from google.cloud import storage

In [ ]:
!pwd

# RAW EXAMPLE

Example taken from the simplfied database on : 
https://console.cloud.google.com/storage/browserquickdraw_dataset/full/raw

In [ ]:
current_dir = os.getcwd()

data_path = os.path.join(os.path.join(os.path.join(current_dir,'..'),'raw_data'),'face.ndjson')

In [ ]:
data_path

In [ ]:
#file_path = "/home/shard777/code/rs-uk/pictionary-ai/raw_data/face.ndjson"

In [ ]:
# Take too much time - DONT DO IT Friends!
#df = pd.read_json(file, lines = True)


In [ ]:
#RAW DATA: Example 
df = pd.DataFrame()
with open(data_path, 'r') as f:
     for i, line in enumerate(f):
            if i > 0:
                break

            json_line = json.loads(line)
            df = pd.concat([df, pd.DataFrame(json_line)], ignore_index=True)
#             print(type(json_line))
#             print(json_line.keys())
#             print(json_line['word'], len(json_line['drawing']))
            coords = json_line['drawing']
    
            for vector in coords:
                print(len(vector))
                print(vector)
           

In [ ]:
df

In [ ]:
df["drawing"][0][0:1]

In [ ]:
drawing_test= df["drawing"][0][2]
drawing_test

In [ ]:

# Extract data from the first row
first_row = df.loc[0]  # Change the index if needed

# Extract x, y, and time lists
x_values = first_row['drawing'][0]
y_values = first_row['drawing'][1]

# Create a scatter plot of the points
plt.scatter(x_values, y_values, color='blue', marker='o', label='Points')

# Connect the points with lines
for i in range(len(x_values) - 1):
    plt.plot([x_values[i], x_values[i + 1]], [y_values[i], y_values[i + 1]], color='red')

# Customize the plot (add labels, title, etc.)
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')
plt.title('Drawing from First RAW Row')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()


In [ ]:
# Extract data from the first drawing 



# Create a new figure for the combined drawing
plt.figure(figsize=(8, 6))

# Iterate through each row and plot the points
num_rows = df.shape[0]
for i in range(num_rows):
    x_values = df.loc[i, 'drawing'][0]
    y_values = df.loc[i, 'drawing'][1]
    plt.plot(x_values, y_values, label=f"Row {i}")

    # Create a scatter plot of the points
    plt.scatter(x_values, y_values, color='blue', marker='o', label='Points')

# Connect the points with lines
for i in range(len(x_values) - 1):
    plt.plot([x_values[i], x_values[i + 1]], [y_values[i], y_values[i + 1]], color='red')

# Customize the plot (add labels, title, etc.)
plt.xlabel('X-coordinate')
plt.ylabel('Y-coordinate')
plt.title('Drawing from First RAW drawing')
plt.grid(True)
plt.legend()

# Show the plot
plt.show()

# SIMPLIFIED EXAMPLE

Example taken from the simplfied database on :
https://console.cloud.google.com/storage/browser/quickdraw_dataset/full/simplified

In [ ]:
#command to download the examle in terminal: gsutil -m cp 'gs://quickdraw_dataset/full/simplified/axe.ndjson' .

In [ ]:
# # Download this face file (simplified) and rename it full_simplified_face:
# gsutil -m cp 'gs://quickdraw_dataset/full/simplified/face.ndjson' full_simplified_face.ndjson


In [23]:
path_data = '/Users/loicregne/code/rs-uk/raw_data'
file_name = 'full_simplified_face.ndjson'

In [25]:
drawing_test2 = df2
drawing_test2

,word,countrycode,timestamp,recognized,key_id,drawing
0,face,CH,2017-03-19 15:52:39.229 UTC,True,5152530285723648,"[[59, 44, 28, 17, 8, 0, 0, 13, 32, 49, 85, 110..."
1,face,CH,2017-03-19 15:52:39.229 UTC,True,5152530285723648,"[[76, 76, 71, 59, 56, 56, 61, 72, 83, 89, 86, ..."
2,face,CH,2017-03-19 15:52:39.229 UTC,True,5152530285723648,"[[176, 166, 146, 133, 130, 157, 192, 198, 196,..."
3,face,CH,2017-03-19 15:52:39.229 UTC,True,5152530285723648,"[[57, 68, 80, 110, 153, 202, 216, 228, 231], [..."


In [111]:
drawing_test2['drawing'][0]

[[59,
  44,
  28,
  17,
  8,
  0,
  0,
  13,
  32,
  49,
  85,
  110,
  136,
  167,
  189,
  209,
  229,
  254,
  254,
  238,
  222,
  206,
  181,
  137,
  86,
  50,
  38],
 [1,
  5,
  20,
  42,
  70,
  109,
  139,
  169,
  190,
  201,
  216,
  221,
  222,
  222,
  217,
  209,
  195,
  144,
  105,
  66,
  48,
  37,
  24,
  8,
  0,
  0,
  7]]

# Processing Simplified file

In [79]:
##Define the function to process the Simplified file and return an array and class name (the drawing to guess)
def parse_line(json_drawing: json) -> dict:
    """Parse an ndjson line and return ink (as np array) and classname."""
    class_name = json_drawing["word"]
    inkarray = json_drawing["drawing"]
    stroke_lengths = [len(stroke[0]) for stroke in inkarray]
    total_points = sum(stroke_lengths)
    np_ink = np.zeros((total_points, 3), dtype=np.float32)
    current_t = 0
    for stroke in inkarray:
        for i in [0, 1]:
            np_ink[current_t:(current_t + len(stroke[0])), i] = stroke[i]
        current_t += len(stroke[0])
        np_ink[current_t - 1, 2] = 1  # stroke_end

    # Preprocessing.
    # 1. Size normalization.
    lower = np.min(np_ink[:, 0:2], axis=0)
    upper = np.max(np_ink[:, 0:2], axis=0)
    scale = upper - lower
    scale[scale == 0] = 1
    np_ink[:, 0:2] = (np_ink[:, 0:2] - lower) / scale

    # 2. Compute deltas.
    np_ink[1:, 0:2] -= np_ink[0:-1, 0:2]
    np_ink = np_ink[1:, :]
    return np.round(np_ink,5), class_name

In [ ]:
##Define the function to process the Simplified file and return an array and class name (the drawing to guess)
def get_drawing_data(json_drawing: json) -> dict:
    '''Extracts the drawing data (strokes list) and class name from a drawing JSON file.'''
    class_name = json_drawing['word']
    lst_strokes = json_drawing['drawing']

    x_values = []
    y_values = []
    stroke_delimiter = []
    
    for stroke in lst_strokes:
        # Creating the third 'column' to pass to the model with 0 all along and a 1 at the end of the stroke
        stroke_delimiter = [0] * len(stroke[0])
        stroke_delimiter[-1] = 1
        # Appending the delimiter column to x and y
        stroke.append(stroke_delimiter)
        lst_strokes_augmented.append(stroke)


    
    lst_strokes_length = [len(stroke[0]) for stroke in lst_inkarray]

    
    total_points = sum(lst_strokes_length)

    
    np_ink = np.zeros((total_points, 3), dtype=np.float32)
    current_t = 0
    for stroke in inkarray:
        for i in [0, 1]:
            np_ink[current_t:(current_t + len(stroke[0])), i] = stroke[i]
        current_t += len(stroke[0])
        np_ink[current_t - 1, 2] = 1  # stroke_end

    # Preprocessing.
    # 1. Size normalization.
    lower = np.min(np_ink[:, 0:2], axis=0)
    upper = np.max(np_ink[:, 0:2], axis=0)
    scale = upper - lower
    scale[scale == 0] = 1
    np_ink[:, 0:2] = (np_ink[:, 0:2] - lower) / scale

    # 2. Compute deltas.
    np_ink[1:, 0:2] -= np_ink[0:-1, 0:2]
    np_ink = np_ink[1:, :]
    return np.round(np_ink,5), class_name

# Define the function to process a  CLASS (one of the word to guess) full file 

In [96]:
ndjson_path = path_data + '/' + file_name

def drawing_extraction(ndjson_path, nb_drawings_to_load):
    dict_drawings = {}  # Initialize an empty dictionary
    with open(ndjson_path, 'r') as f:
        for i, line in enumerate(f):
            json_line = json.loads(line)
            np_ink, class_name = parse_line(json_line)
            #np_ink= np.round(np_ink,5)
            #print(np_ink)
            length_np_ink = len(np_ink)
            dict_drawing_info = {
                "class": class_name,
                "drawing_nb": i,
                "length_np_ink": length_np_ink,
                "np_ink": np_ink.tolist()  #need to be transformed to list to dump as Json file later

            }
            #print(image_info["np_ink"])
            dict_drawings[json_line["key_id"]] = dict_drawing_info
            i += 1
            if i == nb_drawings_to_load:
                break
    return dict_drawings

In [106]:
# Trying a more efficient way to load the data

def drawing_extraction2(ndjson_path, nb_drawings_to_load):
    dict_drawings = {}  # Initialize an empty dictionary

    for i in range(1, nb_drawings_to_load+1):
        json_drawing = json.loads(linecache.getline(ndjson_path, i, module_globals=None))
        np_ink, class_name = parse_line(json_line)
        length_np_ink = len(np_ink)
        dict_drawing_info = {"class": class_name,
                             "drawing_nb": i,
                             "length_np_ink": length_np_ink,
                             "np_ink": np_ink.tolist()  # need to be transformed to list to dump as Json file later
                            }
        #print(image_info["np_ink"])
        dict_drawings[json_line["key_id"]] = dict_drawing_info
        
    return dict_drawings

In [101]:
ndjson_path

'/Users/loicregne/code/rs-uk/raw_data/full_simplified_face.ndjson'

In [107]:
%%time
dict_test = drawing_extraction(ndjson_path, 50000)

CPU times: user 2.6 s, sys: 394 ms, total: 2.99 s
Wall time: 3.06 s


In [108]:
%%time
dict_test = drawing_extraction2(ndjson_path, 50000)

CPU times: user 1.71 s, sys: 50.4 ms, total: 1.76 s
Wall time: 1.76 s


In [86]:
len(dict_test)

161666

In [88]:
# Where we want to save the output file and a name is given
output_file = path_data + '/' + 'processed_full_simplified_face.json'

In [89]:
# Function to save the dictionary file for the processed images
def save_images_dict_to_json(images_dict, output_file):
    with open(output_file, 'w') as json_file:
        json.dump(images_dict, json_file)

In [90]:
save_images_dict_to_json(dict_test,output_file)

In [ ]:
len(dict_test)

In [ ]:
with open(output_file, 'r') as f:
     for i, line in enumerate(f):
        json_line = json.loads(line)
        print(json_line)
        #test_line = parse_line(json_line)
        #print(test_line)
        break

In [115]:
def upload_blob(bucket_name, file_path, file_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    blob.upload_from_filename(file_path + '/' + file_name)

In [116]:
bucket_name = 'quickdraw-simplified-processed'
file_path = path_data
file_name = 'processed_full_simplified_face.json'

upload_blob(bucket_name, file_path, file_name)

Forbidden: 403 POST https://storage.googleapis.com/upload/storage/v1/b/quickdraw-simplified-processed/o?uploadType=resumable: {
  "error": {
    "code": 403,
    "message": "loic-datascience-bootcamp@triple-hour-411312.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
    "errors": [
      {
        "message": "loic-datascience-bootcamp@triple-hour-411312.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object. Permission 'storage.objects.create' denied on resource (or it may not exist).",
        "domain": "global",
        "reason": "forbidden"
      }
    ]
  }
}
: ('Request failed with status code', 403, 'Expected one of', <HTTPStatus.OK: 200>, <HTTPStatus.CREATED: 201>)